In [1]:
# Import important libraries
!pip install requests bs4 selenium webdriver-manager pandas sqlalchemy psycopg2-binary ipython-sql prettytable==3.9.0

In [2]:
# Pulling an API: 

# Step One:  Import the requests library
import requests

In [3]:
# Step Two:  Use the requests library to pull data from an API based on specified parameters (checkout CoinGecko API documentation for more details)
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 10,
    "page": 1,
    "sparkline": "false"
}

response = requests.get(url, params=params)
data = response.json()
print(data)

[{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 110827, 'market_cap': 2209723518768, 'market_cap_rank': 1, 'fully_diluted_valuation': 2209792791912, 'total_volume': 61643929583, 'high_24h': 111555, 'low_24h': 107542, 'price_change_24h': 1873.04, 'price_change_percentage_24h': 1.71912, 'market_cap_change_24h': 37686911847, 'market_cap_change_percentage_24h': 1.7351, 'circulating_supply': 19936690.0, 'total_supply': 19937315.0, 'max_supply': 21000000.0, 'ath': 126080, 'ath_change_percentage': -12.03405, 'ath_date': '2025-10-06T18:57:42.558Z', 'atl': 67.81, 'atl_change_percentage': 163458.48929, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2025-10-20T19:56:00.624Z'}, {'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?1696501628', 'current_price': 3984.2, 'market_cap': 

In [4]:
# Step Three:  Convert the data into a pandas DataFrame and select specific columns to display
import pandas as pd

api_df = pd.DataFrame(data)[['id','symbol', 'current_price', 'market_cap', 'price_change_percentage_24h']]
api_df['source'] = 'CoinGeckoAPI'
api_df

,id,symbol,current_price,market_cap,price_change_percentage_24h,source
0,bitcoin,btc,110827.000000,2209723518768,1.71912,CoinGeckoAPI
1,ethereum,eth,3984.200000,480971670193,-0.13476,CoinGeckoAPI
2,tether,usdt,1.000000,181948031587,-0.02612,CoinGeckoAPI
3,binancecoin,bnb,1100.820000,153247325494,-1.47775,CoinGeckoAPI
4,ripple,xrp,2.540000,152124096532,5.72597,CoinGeckoAPI
5,solana,sol,188.930000,103255420537,0.14163,CoinGeckoAPI
6,usd-coin,usdc,0.999843,75970519269,-0.00495,CoinGeckoAPI
7,staked-ether,steth,3981.190000,33804089036,-0.08164,CoinGeckoAPI
8,tron,trx,0.322251,30506721477,0.55356,CoinGeckoAPI
9,dogecoin,doge,0.199896,30265364266,1.58533,CoinGeckoAPI


In [5]:
# Example of navigating nested JSON reponses from an API
new_response = {
    "data": {
        "layer_one": {
            "layer_two": {
                "name": "BitCoin",
                "symbol": "BTC",
                "price": 30000
            }
        }
    }
}

# Navigating the nested JSON to get to the desired data
finance_data = new_response['data']['layer_one']['layer_two']
finance_data

# Converting the extracted data into a pandas DataFrame
new_dataframe = pd.DataFrame([finance_data])
new_dataframe

,name,symbol,price
0,BitCoin,BTC,30000


In [6]:
# Static Web Scraping

# Step One:  Import the BeautifulSoup library
from bs4 import BeautifulSoup

# Step Two:  Use requests to pull the HTML content of a webpage
url = "https://finance.yahoo.com/cryptocurrencies"
headers = {'User-Agent': 'Mozilla/5.0'} # Some websites block requests that don't have a user-agent
reponse = requests.get(url, headers=headers)
soup = BeautifulSoup(reponse.text, 'html.parser')

soup

<!DOCTYPE html>

<html class="desktop neo-green dock-upscale" data-color-scheme="auto" data-color-theme-enabled="true" lang="en-US" theme="auto">
<head>
<meta charset="utf-8"/>
<meta content="guce.yahoo.com" name="oath:guce:consent-host"/>
<link crossorigin="anonymous" href="//s.yimg.com" rel="preconnect"/><link href="//geo.yahoo.com" rel="preconnect"><link href="//query1.finance.yahoo.com" rel="preconnect"/><link href="//query2.finance.yahoo.com" rel="preconnect"/><link href="//consent.cmp.oath.com" rel="preconnect"/><link as="worker" href="/__rapidworker-1.2.js" rel="preload"/><link href="https://s.yimg.com/uc/finance/webcore/js/_staticFinProtobuf.adb9bf9ae7922cf2dccf.js" rel="preconnect"/><link href="https://cdn.jsdelivr.net/npm/protobufjs@7.1.2/dist/minimal/protobuf.min.js" rel="preconnect"/><script>window.finNeoPageStart = Date.now();</script><script>(function(){if(!window.YAHOO){window.YAHOO={}}; window.YAHOO.context={"consent":{"allowContentPersonalization":true,"allowCrossDevic

In [7]:
# Step Three:  Parse the HTML to extract specific data points and convert them into a pandas DataFrame. Make sure to inspect the webpage to identify the correct HTML tags and classes.
rows = soup.select("table tbody tr")

# Step Four:  Loop through the rows and extract relevant data. Be sure to adjust the indices based on the actual table structure.
data = []
for row in rows[:10]:
    cols = row.find_all('td')
    if len(cols) >= 3:
        name = cols[1].text.strip().split(" ")[0]
        price = cols[3].text.strip().split(" ")[0]
        symbol = cols[0].text.strip().split("-")[0]
        data.append({
            "name": name,
            "price": price,
            "symbol": symbol,
            "source": "Yahoo Finance"
        })

# Step Five:  Convert the extracted data into a pandas DataFrame
yahoo_static_df = pd.DataFrame(data)
yahoo_static_df.head()

,name,price,symbol,source
0,Bitcoin,"110,871.54",B BTC,Yahoo Finance
1,Ethereum,"3,984.10",E ETH,Yahoo Finance
2,Tether,1.00,U USDT,Yahoo Finance
3,BNB,"1,101.71",B BNB,Yahoo Finance
4,XRP,2.54,X XRP,Yahoo Finance


In [8]:
# Debugging tip: Always inspect the HTML structure to ensure you are targeting the correct tags and classes. Trying something like this can help understand what your selectors are pulling:
rows[0].find_all('td')[3].text.strip().split(" ")[0]

'110,871.54'

In [9]:
# Sometimes, if you've created a driver before, creating a new driver might reference old caches and return an error message: "WebDriverException: Can not connect to the Service ...". Run this to clear cache: !rm -rf ~/.wdm

In [10]:
# Dynamic Web Scraping

# The major difference between static and dynamic web scraping is that dynamic web scraping requires a tool that can interact with the webpage as a user would. This is typically done using a web driver like Selenium.
# This method is useful for websites that load content dynamically using JavaScript. This way, you can ensure that all content is fully loaded before you attempt to scrape it.

# Step One:  Import the necessary Selenium libraries and set up the web driver

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Step Two:  Set up the Chrome driver with options
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [11]:
import time
# Step Three:  Use the driver to navigate to the desired webpage and wait for it to load completely
url = "https://coinmarketcap.com/"
driver.get(url)
time.sleep(5)

# Step Four:  Once the page is fully loaded, extract the page source and parse it with BeautifulSoup
response = driver.page_source
soup = BeautifulSoup(response, "html.parser")

soup

<html dir="ltr" lang="en"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, shrink-to-fit=no" name="viewport"/><meta content="https://coinmarketcap.com/" property="og:url"/><link href="https://coinmarketcap.com/" rel="canonical"/><link href="https://coinmarketcap.com/ar/" hreflang="ar" rel="alternate"/><link href="https://coinmarketcap.com/bg/" hreflang="bg" rel="alternate"/><link href="https://coinmarketcap.com/cs/" hreflang="cs" rel="alternate"/><link href="https://coinmarketcap.com/da/" hreflang="da" rel="alternate"/><link href="https://coinmarketcap.com/de/" hreflang="de" rel="alternate"/><link href="https://coinmarketcap.com/el/" hreflang="el" rel="alternate"/><link href="https://coinmarketcap.com/" hreflang="en" rel="alternate"/><link href="https://coinmarketcap.com/es/" hreflang="es" rel="alternate"/><link href="https://coinmarketcap.com/fi/" hreflang="fi" rel=

In [12]:
# Step Five:  Parse the HTML to extract specific data points. Make sure to inspect the webpage to identify the correct HTML tags and classes.
rows = soup.find("tbody").find_all("tr")

# Step Six: Loop through the rows and extract relevant data. Be sure to adjust the indices based on the actual table structure.
data = []
for row in rows[:10]:  # Top 10
    cols = row.find_all("td")
    if len(cols) > 4:
        symbol = cols[2].find("p", class_="coin-item-symbol").text.strip() if cols[2].find("p", class_="coin-item-symbol") else None
        name = cols[2].find("p", class_="coin-item-name").text.strip() if cols[2].find("p") else None
        price = cols[3].text.strip()
        data.append({
            "name": name,
            "symbol": symbol,
            "price": price,
            "source": "CoinMarketCap"
        })

In [13]:
# Step Seven: Convert results to DataFrame
cmc_dynamic_df = pd.DataFrame(data)
cmc_dynamic_df.head()

,name,symbol,price,source
0,Bitcoin,BTC,"$110,871.53",CoinMarketCap
1,Ethereum,ETH,"$3,984.09",CoinMarketCap
2,Tether,USDT,$1.00,CoinMarketCap
3,BNB,BNB,"$1,102.26",CoinMarketCap
4,XRP,XRP,$2.53,CoinMarketCap


In [14]:
# Clean the static & dynamic data to remove the "$" and "," from the amounts in the scrapped tables, and change the symbol on the API dataframe to upper case
yahoo_static_df['price'] = yahoo_static_df['price'].str.replace('[$,]', '', regex=True).astype(float)
cmc_dynamic_df['price'] = cmc_dynamic_df['price'].str.replace('[$,]', '', regex=True).astype(float)
api_df['symbol'] = api_df['symbol'].str.upper()

# Combine all sources into one big DataFrame
combined_df = pd.concat([
    api_df.rename(columns={'id': 'name', 'current_price': 'price'})[['name', 'price', 'source','symbol']],
    yahoo_static_df,
    cmc_dynamic_df
], ignore_index=True)

combined_df

,name,price,source,symbol
0,bitcoin,110827.000000,CoinGeckoAPI,BTC
1,ethereum,3984.200000,CoinGeckoAPI,ETH
2,tether,1.000000,CoinGeckoAPI,USDT
3,binancecoin,1100.820000,CoinGeckoAPI,BNB
4,ripple,2.540000,CoinGeckoAPI,XRP
5,solana,188.930000,CoinGeckoAPI,SOL
6,usd-coin,0.999843,CoinGeckoAPI,USDC
7,staked-ether,3981.190000,CoinGeckoAPI,STETH
8,tron,0.322251,CoinGeckoAPI,TRX
9,dogecoin,0.199896,CoinGeckoAPI,DOGE


In [ ]:
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, MetaData

# Define your PostgreSQL database credentials
db_user = 'postgres'
db_password = 'password'
db_host = 'localhost'
db_port = '5432'
db_name = 'finsight_db'

# Create the database engine and load the data into PostgreSQL
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


In [19]:
# Create the table on Postgres using SQL alchemy. Define table schema with primary key
metadata = MetaData()
market_data = Table(
    "market_data", metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
    Column("price", Float),
    Column("source", String),
    Column("symbol", String)
)

# Create the table in the database
metadata.create_all(engine)

In [21]:
# Create ID Column
combined_df['id'] = combined_df.index + 1

# Move the final DataFrame to PostgreSQL
combined_df.to_sql('market_data', engine, if_exists='replace', index=False)
print("Data successfully loaded into PostgreSQL!")

Data successfully loaded into PostgreSQL!
